In [1]:
import numpy as np
from sentence_transformers import SentenceTransformer
from IPython.display import clear_output
import time

class CategoryScorer:
    def __init__(self):
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        
        self.max_description_length = 20
        
        # Define categories and initial descriptions
        self.category_data = {
            "Sleep": [
                "Going to sleep", 
                "Taking a nap", 
                "Heading to bed", 
                "Resting my eyes", 
                "Unconscious", 
                "Catching some Zs"
            ],
            "Break": [
                "Taking a short break", 
                "Pausing work for a bit", 
                "Relaxing for a moment", 
                "Procrastinating", 
                "Chilling out", 
                "Stopping to rest"
            ],
            "Coding": [
                "Writing code", 
                "Debugging an application", 
                "Programming software", 
                "Developing in Python or Swift", 
                "Building a feature", 
                "Fixing a bug"
            ],
            "Errands": [
                "Doing household chores", 
                "Doing laundry", 
                "Washing the dishes", 
                "Cleaning the house", 
                "Grocery shopping", 
                "Tidying up the room"
            ],
            "Fitness": [
                "Working out", 
                "Going for a run", 
                "Lifting weights at the gym", 
                "Doing cardio", 
                "Playing sports", 
                "Exercising"
            ],
            "Meditation": [
                "Meditating", 
                "Practicing mindfulness", 
                "Doing deep breathing exercises", 
                "Yoga session", 
                "Sitting quietly"
            ],
            "Study": [
                "Studying for an exam", 
                "Doing homework", 
                "Reading a textbook", 
                "Writing an essay", 
                "Learning a new subject", 
                "Classwork"
            ],
            "Work": [
                "Working at my job", 
                "Sitting at my desk working", 
                "In a meeting", 
                "Answering emails", 
                "Professional business tasks", 
                "Career work"
            ],
            "Leisure": [
                "Watching TV", 
                "Playing video games", 
                "Watching a movie", 
                "Scroll social media", 
                "Having fun", 
                "Hobby time", 
                "Relaxing on the couch"
            ],
            "Eat": [
                "Eating a meal", 
                "Having breakfast", 
                "Grabbing lunch", 
                "Eating dinner", 
                "Having a snack", 
                "Drinking water"
            ],
            "Commute": [
                "Commuting to work", 
                "Driving to the office", 
                "Walking to school", 
                "Heading to a destination", 
                "Taking the bus or train", 
                "Riding a bike", 
                "Traveling",
                "Going to work" 
            ]
        }
        
        # Split category descriptions
        for category, descriptions in self.category_data.items():
            if len(descriptions) == 1 and "," in descriptions[0]:
                clean_list = [d.strip() for d in descriptions[0].split(',') if d.strip()]
                self.category_data[category] = clean_list
        
        # Pre-calculate vectors for existing categories
        self.category_vectors = {}
        self.calculate_all_category_vectors()

    def predict(self, text):
        similarities = []
        
        # Get vector embedding of input text
        text_embedding = self.model.encode(text, normalize_embeddings=True)
        
        # Get words in input text
        text_words = set(text.lower().split())
        
        # Compare input vector with all category vectors
        for category, category_embedding in self.category_vectors.items():    
            vector_score = np.dot(text_embedding, category_embedding)
            
            description_words = set(self.get_full_description(category))
            intersection = len(text_words.intersection(description_words))
            keyword_boost = 0.1 * intersection
            
            final_score = vector_score + keyword_boost
            
            similarities.append((category, final_score))
            
        # Sort similarities by score
        similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
            
        return similarities
    
    def get_full_description(self, category):
        return [category] + self.category_data[category]
    
    def get_category_vector(self, category):
        full_description = self.get_full_description(category)
        embeddings = self.model.encode(full_description, normalize_embeddings=True)
        embeddings[0, :] *= 3
        
        centroid = np.mean(embeddings, axis=0)
        normalized_centroid = centroid / np.linalg.norm(centroid)
        
        return normalized_centroid
    
    def calculate_all_category_vectors(self):
        for category in self.category_data:
            self.category_vectors[category] = self.get_category_vector(category)
            
    def update_category(self, category, text):
        if category not in self.category_data:
            self.category_data[category] = []
        
        self.category_data[category].append(text)
        
        if len(self.category_data[category]) > self.max_description_length:
            self.category_data[category] = self.category_data[category][1:]
        
        self.category_vectors[category] = self.get_category_vector(category)
        
# --- THE INTERACTIVE LOOP ---

scorer = CategoryScorer()

while True:
    print("-" * 60)
    user_input = input("Enter activity (or 'q' to quit): ")
    
    if user_input.lower() in ['q', 'quit', 'exit']:
        print("Goodbye!")
        break
        
    if not user_input.strip():
        continue

    # 1. Get Predictions
    predictions = scorer.predict(user_input)
    top_category = predictions[0][0]
    top_score = predictions[0][1]

    # 2. Display Dashboard
    print(f"\n📝 Input: '{user_input}'")
    print(f"🤖 Top Prediction: {top_category} ({top_score:.3f})\n")
    
    print("Scores:")
    for i, (cat, score) in enumerate(predictions[:10]): # Show Top 10
        bar = "█" * int(score * 20)
        print(f"  [{i}] {cat:<12} {score:.3f}  {bar}")
    
    # 3. User Feedback Loop
    print("\nActions:")
    print("  [Enter] Confirm Top Match")
    print("  [0-9]   Select specific category above")
    print("  [n]     Create NEW Category")
    
    choice = input("Select correct category: ").strip().lower()
    
    selected_category = None
    
    # CASE A: Confirm Top Match
    if choice == "":
        selected_category = top_category
        # Only learn if confidence was low, otherwise skip to save space (Optional rule)
        if top_score < 0.8: 
            scorer.update_category(selected_category, user_input)
        else:
            print("✅ High confidence match. No update needed.")

    # CASE B: Select from List
    elif choice.isdigit() and 0 <= int(choice) < 10:
        idx = int(choice)
        selected_category = predictions[idx][0]
        # This is a correction, so we ALWAYS update
        scorer.update_category(selected_category, user_input)

    # CASE C: Create New Category
    elif choice == 'n':
        new_cat_name = input("Enter name for NEW category: ").strip().title()
        if new_cat_name:
            selected_category = new_cat_name
            # Initialize with user input
            scorer.update_category(selected_category, user_input)

    # CASE D: Invalid
    else:
        print("❌ Invalid selection. Learning skipped.")

    # Pause for effect so user can read result
    time.sleep(1)

/Users/mac-aroni/miniforge3/envs/time-tracker-ml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


------------------------------------------------------------
Goodbye!


In [2]:
import numpy as np
from sentence_transformers import SentenceTransformer
from IPython.display import clear_output
import time

class CategoryScorer:
    def __init__(self):
        self.model = SentenceTransformer("all-mpnet-base-v2")
        
        self.max_description_length = 20
        
        # Define categories and initial descriptions
        self.category_data = {
            "Sleep": ["Sleep, take a nap, go to bed, rest eyes, unconscious."],
            "Break": ["Take a break, pause work, relax for a moment, procrastination, chill."],
            "Coding": ["Write code, programming, software development, debugging, python, swift."],
            "Errands": ["Do chores, laundry, wash dishes, clean house, buy groceries, housework."],
            "Fitness": ["Workout, go to the gym, run, lift weights, exercise, cardio, sports."],
            "Meditation": ["Meditate, mindfulness, deep breathing, yoga, sit quietly."],
            "Study": ["Study for school, do homework, read textbook, learn new things, class work."],
            "Work": ["Do my job, work at office, professional tasks, business, career, meetings."],
            "Leisure": ["Watch TV, watch a movie, play games, entertainment, hobby, fun, relax."],
            "Eat": ["Eat a meal, have breakfast, lunch, dinner, snack, drink water."],
            "Commute": ["Commute, travel, drive car, walk to place, take bus, train, ride bike."],
        }
        
        # Split category descriptions
        for category, descriptions in self.category_data.items():
            if len(descriptions) == 1 and "," in descriptions[0]:
                clean_list = [d.strip() for d in descriptions[0].split(',') if d.strip()]
                self.category_data[category] = clean_list
        
        # Pre-calculate vectors for existing categories
        self.category_vectors = {}
        self.calculate_all_category_vectors()

    def predict(self, text):
        similarities = []
        
        # Get vector embedding of input text
        text_embedding = self.model.encode(text, normalize_embeddings=True)
        
        # Compare input vector with all category vectors
        for category, category_embedding in self.category_vectors.items():
            similarities.append((category, np.dot(text_embedding, category_embedding)))
            
        # Sort similarities by score
        similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
            
        return similarities
    
    def get_category_vector(self, category):
        full_description = [category] + self.category_data[category]
        embeddings = self.model.encode(full_description, normalize_embeddings=True)
        centroid = np.mean(embeddings, axis=0)
        normalized_centroid = centroid / np.linalg.norm(centroid)
        
        return normalized_centroid
    
    def calculate_all_category_vectors(self):
        for category in self.category_data:
            self.category_vectors[category] = self.get_category_vector(category)
            
    def update_category(self, category, text):
        if category not in self.category_data:
            self.category_data[category] = []
        
        self.category_data[category].append(text)
        
        if len(self.category_data[category]) > self.max_description_length:
            self.category_data[category] = self.category_data[category][1:]
        
        self.category_vectors[category] = self.get_category_vector(category)
        
# --- THE INTERACTIVE LOOP ---

scorer = CategoryScorer()

while True:
    print("-" * 60)
    user_input = input("Enter activity (or 'q' to quit): ")
    
    if user_input.lower() in ['q', 'quit', 'exit']:
        print("Goodbye!")
        break
        
    if not user_input.strip():
        continue

    # 1. Get Predictions
    predictions = scorer.predict(user_input)
    top_category = predictions[0][0]
    top_score = predictions[0][1]

    # 2. Display Dashboard
    print(f"\n📝 Input: '{user_input}'")
    print(f"🤖 Top Prediction: {top_category} ({top_score:.3f})\n")
    
    print("Scores:")
    for i, (cat, score) in enumerate(predictions[:10]): # Show Top 10
        bar = "█" * int(score * 20)
        print(f"  [{i}] {cat:<12} {score:.3f}  {bar}")
    
    # 3. User Feedback Loop
    print("\nActions:")
    print("  [Enter] Confirm Top Match")
    print("  [0-9]   Select specific category above")
    print("  [n]     Create NEW Category")
    
    choice = input("Select correct category: ").strip().lower()
    
    selected_category = None
    
    # CASE A: Confirm Top Match
    if choice == "":
        selected_category = top_category
        # Only learn if confidence was low, otherwise skip to save space (Optional rule)
        if top_score < 0.8: 
            scorer.update_category(selected_category, user_input)
        else:
            print("✅ High confidence match. No update needed.")

    # CASE B: Select from List
    elif choice.isdigit() and 0 <= int(choice) < 10:
        idx = int(choice)
        selected_category = predictions[idx][0]
        # This is a correction, so we ALWAYS update
        scorer.update_category(selected_category, user_input)

    # CASE C: Create New Category
    elif choice == 'n':
        new_cat_name = input("Enter name for NEW category: ").strip().title()
        if new_cat_name:
            selected_category = new_cat_name
            # Initialize with user input
            scorer.update_category(selected_category, user_input)

    # CASE D: Invalid
    else:
        print("❌ Invalid selection. Learning skipped.")

    # Pause for effect so user can read result
    time.sleep(1)

------------------------------------------------------------
Goodbye!


In [1]:
data = {
    "Sleep": [
        "Going to sleep", 
        "Taking a nap", 
        "Heading to bed", 
        "Resting my eyes", 
        "Unconscious", 
        "Catching some Zs"
    ],
    "Break": [
        "Taking a short break", 
        "Pausing work for a bit", 
        "Relaxing for a moment", 
        "Procrastinating", 
        "Chilling out", 
        "Stopping to rest"
    ],
    "Coding": [
        "Writing code", 
        "Debugging an application", 
        "Programming software", 
        "Developing in Python or Swift", 
        "Building a feature", 
        "Fixing a bug"
    ],
    "Errands": [
        "Doing household chores", 
        "Doing laundry", 
        "Washing the dishes", 
        "Cleaning the house", 
        "Grocery shopping", 
        "Tidying up the room"
    ],
    "Fitness": [
        "Working out", 
        "Going for a run", 
        "Lifting weights at the gym", 
        "Doing cardio", 
        "Playing sports", 
        "Exercising"
    ],
    "Meditation": [
        "Meditating", 
        "Practicing mindfulness", 
        "Doing deep breathing exercises", 
        "Yoga session", 
        "Sitting quietly"
    ],
    "Study": [
        "Studying for an exam", 
        "Doing homework", 
        "Reading a textbook", 
        "Writing an essay", 
        "Learning a new subject", 
        "Classwork"
    ],
    "Work": [
        "Working at my job", 
        "Sitting at my desk working", 
        "In a meeting", 
        "Answering emails", 
        "Professional business tasks", 
        "Career work"
    ],
    "Leisure": [
        "Watching TV", 
        "Playing video games", 
        "Watching a movie", 
        "Scroll social media", 
        "Having fun", 
        "Hobby time", 
        "Relaxing on the couch"
    ],
    "Eat": [
        "Eating a meal", 
        "Having breakfast", 
        "Grabbing lunch", 
        "Eating dinner", 
        "Having a snack", 
        "Drinking water"
    ],
    "Commute": [
        "Commuting to work", 
        "Driving to the office", 
        "Walking to school", 
        "Heading to a destination", 
        "Taking the bus or train", 
        "Riding a bike", 
        "Traveling",
        "Going to work" 
    ]
}

In [3]:
from scorer import Scorer
import time

model = Scorer(data)
model.initialize_vectors()

while True:
    print("-" * 60)
    input_text = input("Enter activity (or 'q' to quit): ")
    
    if input_text.lower() in ['q', 'quit', 'exit']:
        print("Goodbye!")
        break
        
    if not input_text.strip():
        continue
    
    predictions = model.predict(input_text)
    top_category = predictions[0][0]
    top_score = predictions[0][1]
    
    print(top_score, type(top_score))

    # 2. Display Dashboard
    print(f"\n📝 Input: '{input_text}'")
    print(f"🤖 Top Prediction: {top_category} ({top_score:.3f})\n")
    
    print("Scores:")
    for i, (cat, score) in enumerate(predictions[:10]): # Show Top 10
        bar = "█" * int(score * 20)
        print(f"  [{i}] {cat:<12} {score:.3f}  {bar}")
    
    # 3. User Feedback Loop
    print("\nActions:")
    print("  [Enter] Confirm Top Match")
    print("  [0-9]   Select specific category above")
    print("  [n]     Create NEW Category")
    
    choice = input("Select correct category: ").strip().lower()
    
    selected_category = None
    
    # CASE A: Confirm Top Match
    if choice == "":
        selected_category = top_category
        # Only learn if confidence was low, otherwise skip to save space (Optional rule)
        if top_score < 0.8: 
            model.update_descriptions(selected_category, input_text)
        else:
            print("✅ High confidence match. No update needed.")

    # CASE B: Select from List
    elif choice.isdigit() and 0 <= int(choice) < 10:
        idx = int(choice)
        selected_category = predictions[idx][0]
        # This is a correction, so we ALWAYS update
        model.update_descriptions(selected_category, input_text)

    # CASE C: Create New Category
    elif choice == 'n':
        new_cat_name = input("Enter name for NEW category: ").strip().title()
        if new_cat_name:
            selected_category = new_cat_name
            # Initialize with user input
            model.update_descriptions(selected_category, input_text)

    # CASE D: Invalid
    else:
        print("❌ Invalid selection. Learning skipped.")

    # Pause for effect so user can read result
    time.sleep(1)

------------------------------------------------------------
0.6095359921455383 <class 'float'>

📝 Input: 'watch instagram'
🤖 Top Prediction: Leisure (0.610)

Scores:
  [0] Leisure      0.610  ████████████
  [1] Eat          0.482  █████████
  [2] Break        0.470  █████████
  [3] Meditation   0.436  ████████
  [4] Commute      0.417  ████████
  [5] Fitness      0.414  ████████
  [6] Study        0.413  ████████
  [7] Sleep        0.382  ███████
  [8] Coding       0.369  ███████
  [9] Errands      0.368  ███████

Actions:
  [Enter] Confirm Top Match
  [0-9]   Select specific category above
  [n]     Create NEW Category
------------------------------------------------------------
0.5513965487480164 <class 'float'>

📝 Input: 'watching beaver'
🤖 Top Prediction: Leisure (0.551)

Scores:
  [0] Leisure      0.551  ███████████
  [1] Sleep        0.496  █████████
  [2] Study        0.495  █████████
  [3] Eat          0.481  █████████
  [4] Break        0.459  █████████
  [5] Errands      0.4

KeyboardInterrupt: Interrupted by user

In [ ]:
from scorer import Scorer

labeled_tests = [
    # --- SLEEP ---
    ("Nap", "Sleep"), ("Sleeping", "Sleep"), ("Bedtime", "Sleep"), 
    ("Snooze", "Sleep"), ("Dozing", "Sleep"), ("Passed out", "Sleep"), 
    ("Crash", "Sleep"), ("Power nap", "Sleep"), ("Resting eyes", "Sleep"), 
    ("Zzz", "Sleep"), ("Siesta", "Sleep"), ("Out cold", "Sleep"), ("Hibernate", "Sleep"),

    # --- BREAK ---
    ("Break", "Break"), ("Pausing", "Break"), ("Stop", "Break"), 
    ("Chill", "Break"), ("Chilling", "Break"), ("Rest", "Break"), 
    ("Breather", "Break"), ("Timeout", "Break"), ("AFK", "Break"), 
    ("Brb", "Break"), ("5 mins", "Break"), ("Spacing out", "Break"), 
    ("Procrastinating", "Break"),

    # --- CODING ---
    ("Code", "Coding"), ("Coding", "Coding"), ("Dev", "Coding"), 
    ("Programming", "Coding"), ("Hack", "Coding"), ("Python", "Coding"), 
    ("Swift", "Coding"), ("Java", "Coding"), ("C++", "Coding"), 
    ("Debugging", "Coding"), ("Fix bug", "Coding"), ("Bug fix", "Coding"), 
    ("Push to git", "Coding"), ("Commit", "Coding"), ("Merge", "Coding"), 
    ("Refactor", "Coding"), ("Deploy", "Coding"), ("PR review", "Coding"), 
    ("Stackoverflow", "Coding"), ("Terminal", "Coding"),

    # --- ERRANDS ---
    ("Clean", "Errands"), ("Cleaning", "Errands"), ("Tidy", "Errands"), 
    ("Tidying", "Errands"), ("Chores", "Errands"), ("Laundry", "Errands"), 
    ("Wash clothes", "Errands"), ("Fold", "Errands"), ("Ironing", "Errands"), 
    ("Dishes", "Errands"), ("Loading dishwasher", "Errands"), ("Vacuum", "Errands"), 
    ("Vacuuming", "Errands"), ("Sweep", "Errands"), ("Mop", "Errands"), 
    ("Trash", "Errands"), ("Take out garbage", "Errands"), ("Recycling", "Errands"), 
    ("Organize", "Errands"), ("Declutter", "Errands"), ("Groceries", "Errands"), 
    ("Shopping", "Errands"), ("Target run", "Errands"), ("Costco", "Errands"), 
    ("Buy food", "Errands"), ("Water plants", "Errands"), ("Watering", "Errands"), 
    ("Fix sink", "Errands"), ("Change bulb", "Errands"), ("Repairs", "Errands"),

    # --- FITNESS ---
    ("Gym", "Fitness"), ("Workout", "Fitness"), ("Training", "Fitness"), 
    ("Exercise", "Fitness"), ("Lift", "Fitness"), ("Weights", "Fitness"), 
    ("Cardio", "Fitness"), ("Run", "Fitness"), ("Running", "Fitness"), 
    ("Jog", "Fitness"), ("Jogging", "Fitness"), ("Treadmill", "Fitness"), 
    # Note: "Walk" is ambiguous, but usually implies fitness in this context unless specified
    ("Walk dog", "Fitness"), ("Walking the dog", "Fitness"), ("Walk puppy", "Fitness"), 
    ("Hike", "Fitness"), ("Hiking", "Fitness"), ("Bike", "Fitness"), 
    ("Cycling", "Fitness"), ("Spin class", "Fitness"), ("Yoga", "Fitness"), 
    ("Pilates", "Fitness"), ("Stretch", "Fitness"), ("Swim", "Fitness"), 
    ("Swimming", "Fitness"), ("Basketball", "Fitness"), ("Tennis", "Fitness"), 
    ("Soccer", "Fitness"), ("Pushups", "Fitness"), ("Situps", "Fitness"), 
    ("Leg day", "Fitness"), ("Warm up", "Fitness"), ("Cool down", "Fitness"),

    # --- MEDITATION ---
    ("Meditate", "Meditation"), ("Meditation", "Meditation"), ("Zen", "Meditation"), 
    ("Om", "Meditation"), ("Mindful", "Meditation"), ("Mindfulness", "Meditation"), 
    ("Breathe", "Meditation"), ("Breathing", "Meditation"), ("Deep breaths", "Meditation"), 
    ("Silence", "Meditation"), ("Sit", "Meditation"), ("Sitting", "Meditation"), 
    ("Quiet time", "Meditation"), ("Focus", "Meditation"), ("Headspace", "Meditation"),

    # --- STUDY ---
    ("Study", "Study"), ("Studying", "Study"), ("Homework", "Study"), 
    ("HW", "Study"), ("School", "Study"), ("Class", "Study"), 
    ("Lecture", "Study"), ("Read", "Study"), ("Reading", "Study"), 
    ("Textbook", "Study"), ("Notes", "Study"), ("Review", "Study"), 
    ("Cramming", "Study"), ("Essay", "Study"), ("Paper", "Study"), 
    ("Write", "Study"), ("Writing", "Study"), ("Research", "Study"), 
    ("Learn", "Study"), ("Learning", "Study"), ("Exam prep", "Study"), 
    ("Quiz", "Study"), ("Test", "Study"), ("Flashcards", "Study"), ("Anki", "Study"),

    # --- WORK ---
    ("Work", "Work"), ("Working", "Work"), ("Job", "Work"), 
    ("Office", "Work"), ("Shift", "Work"), ("Meeting", "Work"), 
    ("Zoom", "Work"), ("Teams", "Work"), ("Call", "Work"), 
    ("Standup", "Work"), ("Email", "Work"), ("Emails", "Work"), 
    ("Inbox", "Work"), ("Slack", "Work"), ("Msg", "Work"), 
    ("Report", "Work"), ("Reports", "Work"), ("Excel", "Work"), 
    ("Spreadsheet", "Work"), ("Docs", "Work"), ("Presentation", "Work"), 
    ("Slides", "Work"), ("Planning", "Work"), ("Strategy", "Work"), 
    ("Admin", "Work"), ("Taxes", "Work"), ("Paperwork", "Work"), 
    ("Filing", "Work"), ("Billing", "Work"), ("Invoices", "Work"),

    # --- LEISURE ---
    ("TV", "Leisure"), ("Watch TV", "Leisure"), ("Netflix", "Leisure"), 
    ("Hulu", "Leisure"), ("Movie", "Leisure"), ("Film", "Leisure"), 
    ("Show", "Leisure"), ("Series", "Leisure"), ("YouTube", "Leisure"), 
    ("Video", "Leisure"), ("Game", "Leisure"), ("Gaming", "Leisure"), 
    ("Play", "Leisure"), ("PS5", "Leisure"), ("Xbox", "Leisure"), 
    ("Switch", "Leisure"), ("Steam", "Leisure"), ("LoL", "Leisure"), 
    ("COD", "Leisure"), ("Minecraft", "Leisure"), ("Scroll", "Leisure"), 
    ("Scrolling", "Leisure"), ("Insta", "Leisure"), ("Instagram", "Leisure"), 
    ("TikTok", "Leisure"), ("Twitter", "Leisure"), ("X", "Leisure"), 
    ("Reddit", "Leisure"), ("Social media", "Leisure"), ("Phone", "Leisure"), 
    ("Read book", "Leisure"), ("Novel", "Leisure"), ("Kindle", "Leisure"), 
    ("Podcast", "Leisure"), ("Music", "Leisure"), ("Chill on couch", "Leisure"), 
    ("Lounge", "Leisure"), ("Hang out", "Leisure"), ("Relax", "Leisure"), 
    ("Vibing", "Leisure"),

    # --- EAT ---
    ("Eat", "Eat"), ("Eating", "Eat"), ("Food", "Eat"), 
    ("Hungry", "Eat"), ("Meal", "Eat"), ("Breakfast", "Eat"), 
    ("Brekkie", "Eat"), ("Lunch", "Eat"), ("Dinner", "Eat"), 
    ("Supper", "Eat"), ("Snack", "Eat"), ("Snacking", "Eat"), 
    ("Munch", "Eat"), ("Treat", "Eat"), ("Dessert", "Eat"), 
    ("Cook", "Eat"), ("Cooking", "Eat"), ("Bake", "Eat"), 
    ("Baking", "Eat"), ("Prep food", "Eat"), ("Drink", "Eat"), 
    ("Drinking", "Eat"), ("Water", "Eat"), ("Hydrate", "Eat"), 
    ("Coffee", "Eat"), ("Tea", "Eat"), ("Soda", "Eat"), 
    ("Beer", "Eat"), ("Wine", "Eat"), ("Cocktail", "Eat"),

    # --- COMMUTE ---
    ("Commute", "Commute"), ("Commuting", "Commute"), ("Travel", "Commute"), 
    ("Trip", "Commute"), ("Go to work", "Commute"), ("Drive", "Commute"), 
    ("Driving", "Commute"), ("Car", "Commute"), ("Traffic", "Commute"), 
    ("Ride", "Commute"), ("Bus", "Commute"), ("Train", "Commute"), 
    ("Subway", "Commute"), ("Metro", "Commute"), ("Uber", "Commute"), 
    ("Lyft", "Commute"), ("Taxi", "Commute"), ("Flight", "Commute"), 
    ("Fly", "Commute"), ("Plane", "Commute"), ("Bike to work", "Commute"), 
    ("Walk to work", "Commute"), ("Go home", "Commute"), 
    ("Heading home", "Commute"), ("On the way", "Commute"),

    # --- TYPOS ---
    ("slewp", "Sleep"), ("workin", "Work"), ("gymm", "Fitness"), 
    ("studdy", "Study"), ("codign", "Coding"), ("meetng", "Work"), 
    ("laundryy", "Errands"), ("brakfast", "Eat"), ("commte", "Commute"), 
    ("netlfix", "Leisure")
]

# List of models to compete against MiniLM-L6
models_to_test = [
    "BAAI/bge-base-en-v1.5",
    "intfloat/e5-small-v2",
    "sentence-transformers/all-MiniLM-L6-v2",       # Current Champion
]

print(f"{'MODEL':<40} | {'ACCURACY':<8} | {'SCORE':<8} | {'AVG CONF':<8}")
print("-" * 80)

for model_name in models_to_test:
    # Initialize scorer with specific model
    # Note: You might need try/except blocks if some models require trust_remote_code=True
    try:
        model = Scorer(data, model_name)
        
        if "e5-small" in model_name:
            model.query_prefix = "query: "
            model.doc_prefix = "passage: "
        
        model.initialize_vectors()
        
        total_score = 0
        correct_count = 0
        conf_sum = 0
        
        for text, true_label in labeled_tests:
            predictions = model.predict(text)
            top_label, top_score = predictions[0]
            
            conf_sum += top_score
            
            # Find Rank
            labels_only = [p[0] for p in predictions]
            try:
                rank = labels_only.index(true_label) + 1
            except ValueError:
                rank = 100
            
            # Scoring Logic
            if rank == 1:
                total_score += top_score
                correct_count += 1
            else:
                penalty = top_score * (rank - 1)
                total_score -= penalty
        
        accuracy = (correct_count / len(labeled_tests)) * 100
        avg_conf = conf_sum / len(labeled_tests)
        
        print(f"{model_name:<40} | {accuracy:.1f}%   | {total_score:>7.2f}  | {avg_conf:.3f}")
        
    except Exception as e:
        print(f"{model_name:<40} | FAILED TO LOAD: {e}")

MODEL                                    | ACCURACY | SCORE    | AVG CONF
--------------------------------------------------------------------------------
BAAI/bge-base-en-v1.5                    | 78.2%   |   68.89  | 0.678
sentence-transformers/all-MiniLM-L6-v2   | 74.6%   |   57.95  | 0.457
